In [1]:
import os
import math
import numpy as np

import torch
from torchvision import transforms
from torchvision.transforms import functional as TF
from torch.utils.data import DataLoader
from datasets import load_dataset
from tqdm import tqdm
from argparse import Namespace

from GeospatialFM.datasets.GFMBench.utils import get_dataset, get_metadata
from GeospatialFM.data_process import apply_normalization, modal_specific_collate_fn, get_transform
from GeospatialFM.finetune.utils import get_task_model, get_loss_fn
# from GeospatialFM.models import SpatialSpectralLowRankViTConfig, SpatialSpectralMAEViT
# from GeospatialFM.models.low_rank_attention import get_perception_field_mask
# from GeospatialFM.models import PositionalChannelEmbedding
# from GeospatialFM.datasets import SSL4EODataset
# from GeospatialFM.scripts.trainer import MAETrainer

from functools import partial
from accelerate import Accelerator
from transformers import TrainingArguments

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import rasterio
import pandas as pd
import json

%load_ext autoreload
%autoreload 2

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args = Namespace(
    dataset_name = "eurosat",
    task_type = "classification",
    scale = 2,
    data_dir = "/home/haozhesi/Dropbox/GeospatialFM/data/geospatial",
    crop_size = None,
    modal = "optical",
    return_dict = True,
    pretrained_model_path = "/home/haozhesi/Dropbox/GeospatialFM/results/models/LESSVIT_b2_d6/checkpoint-33000/model.safetensors",
    patch_size=16,
    embed_dim=768,
    channel_embed_dims_per_head=2,
    depth=12,
    num_heads=12,
    decoder_embed_dim=512,
    decoder_depth=6,
    decoder_num_heads=16,
    decoder_channel_embed_dims_per_head=2,
    use_perception_field_mask=True,
    attention_radius=320,
    norm_pix_loss=False,
    decoder_out_chans = 15,
    pos_chan_embed_residual=True,
)

In [3]:
metadata = get_metadata(args.dataset_name)
optical_mean, optical_std = metadata["s2c"]["mean"], metadata["s2c"]["std"]
radar_mean, radar_std = metadata["s1"]["mean"], metadata["s1"]["std"]
print(optical_mean, radar_mean)
print(optical_std, radar_std)
print(metadata["size"], metadata['num_classes'])

[1354.40546513, 1118.24399958, 1042.92983953, 947.62620298, 1199.47283961, 1999.79090914, 2369.22292565, 2296.82608323, 732.08340178, 12.11327804, 1819.01027855, 1118.92391149, 2594.14080798] None
[245.71762908, 333.00778264, 395.09249139, 593.75055589, 566.4170017, 861.18399006, 1086.63139075, 1117.98170791, 404.91978886, 4.77584468, 1002.58768311, 761.30323499, 1231.58581042] None
64 10


In [4]:
standard_transform = partial(apply_normalization, optical_mean=optical_mean, optical_std=optical_std, radar_mean=radar_mean, radar_std=radar_std, use_8bit=False)
collate_fn = partial(modal_specific_collate_fn, normalization=standard_transform, modal=args.modal)

In [5]:
crop_size = metadata["size"] if args.crop_size is None else args.crop_size
train_transform, eval_transform = get_transform(args.task_type, crop_size=crop_size, scale=args.scale)


In [6]:
dataset = get_dataset(args, train_transform, eval_transform)

In [7]:
# dataset["train"].set_format(type='torch', columns=['optical', 'label', 'optical_channel_wv', 'spatial_resolution'])
# dataset["train"].set_transform(train_transform)

In [22]:
# len(dataset["train"][0]['optical'][0][0])
dataset['val'][1]

64
64


{'optical': tensor([[[1053.0000, 1052.9561, 1052.8625,  ..., 1095.6182, 1092.8613,
           1091.5588],
          [1053.0219, 1052.9785, 1052.8860,  ..., 1095.7838, 1093.0010,
           1091.6863],
          [1053.0687, 1053.0264, 1052.9360,  ..., 1096.1375, 1093.2996,
           1091.9589],
          ...,
          [1041.5115, 1041.5383, 1041.5953,  ..., 1037.0000, 1037.0000,
           1037.0000],
          [1042.6423, 1042.6820, 1042.7667,  ..., 1037.0000, 1037.0000,
           1037.0000],
          [1043.1764, 1043.2222, 1043.3199,  ..., 1037.0000, 1037.0000,
           1037.0000]],
 
         [[ 739.9999,  740.1533,  740.4810,  ...,  787.5267,  796.2189,
            800.3235],
          [ 740.0657,  740.2214,  740.5541,  ...,  787.5588,  796.2280,
            800.3216],
          [ 740.2061,  740.3669,  740.7107,  ...,  787.6274,  796.2471,
            800.3174],
          ...,
          [ 734.8549,  735.0010,  735.3134,  ...,  734.9686,  742.0842,
            745.4468],
      

In [10]:
dataloader = DataLoader(dataset["train"], batch_size=64, collate_fn=collate_fn)
for i, batch in enumerate(dataloader):
    # print(batch['optical'].shape)
    for key, value in batch.items():
        print(key, value.shape)
    break
labels = batch.pop("labels")

spatial_resolution ()
labels torch.Size([64])
optical torch.Size([64, 13, 128, 128])
optical_channel_wv torch.Size([1, 13])


In [11]:
model = get_task_model(args, metadata["num_classes"], metadata["size"])
if args.pretrained_model_path:
        from safetensors import safe_open
        with safe_open(args.pretrained_model_path, framework="pt", device="cpu") as f:
            # Load only encoder weights
            for key in f.keys():
                if key.startswith("encoder."):
                    # Get the corresponding key in target model
                    param = f.get_tensor(key)
                    model.state_dict()[key].copy_(param)

In [12]:
labels, batch

(tensor([7, 3, 2, 6, 9, 8, 6, 5, 1, 7, 1, 1, 9, 6, 1, 5, 4, 3, 1, 3, 6, 9, 5, 9,
         2, 2, 3, 8, 8, 0, 6, 3, 4, 2, 4, 6, 1, 9, 8, 1, 7, 2, 3, 1, 4, 8, 7, 9,
         3, 7, 3, 5, 2, 0, 1, 8, 2, 1, 4, 9, 7, 4, 5, 5]),
 {'spatial_resolution': array(5.),
  'optical': tensor([[[[0.2900, 0.2900, 0.2899,  ..., 0.3730, 0.3702, 0.3689],
            [0.2900, 0.2900, 0.2899,  ..., 0.3728, 0.3700, 0.3686],
            [0.2900, 0.2900, 0.2899,  ..., 0.3723, 0.3694, 0.3681],
            ...,
            [0.3348, 0.3350, 0.3353,  ..., 0.5222, 0.5243, 0.5253],
            [0.3348, 0.3350, 0.3354,  ..., 0.5262, 0.5283, 0.5293],
            [0.3348, 0.3350, 0.3354,  ..., 0.5281, 0.5301, 0.5311]],
  
           [[0.3054, 0.3053, 0.3050,  ..., 0.3315, 0.3200, 0.3146],
            [0.3052, 0.3051, 0.3049,  ..., 0.3316, 0.3198, 0.3142],
            [0.3049, 0.3048, 0.3045,  ..., 0.3318, 0.3193, 0.3134],
            ...,
            [0.2959, 0.2963, 0.2971,  ..., 0.6335, 0.4611, 0.3797],
            [0.

In [13]:
outputs = model(**batch)

In [60]:
outputs

{'logits': tensor([[-0.0497, -0.6642, -0.4838,  0.3608, -0.5332,  0.9910,  0.4215, -0.8076,
          -0.7764, -0.5394],
         [-0.1339, -0.6321, -0.5368,  0.7132, -0.8973,  0.9228,  0.2891, -0.6939,
          -0.5059, -0.4897]], grad_fn=<AddmmBackward0>)}

In [61]:
loss_fn = get_loss_fn(args.task_type)

In [62]:
loss_fn(outputs, labels)

tensor(2.3406, grad_fn=<NllLossBackward0>)